In [1]:
%pwd
%cd ../..

/home/rydevera3/data-science/umn_research/Deeplifting


In [2]:
import glob

import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME

In [ ]:
# Get the results from deeplifting
files = glob.glob('./results/results-2023-08-03-2*')
deeplifting_results = pd.read_parquet(files)

# Select specific columns for results
deeplifting_results = (
    deeplifting_results[['problem_name', 'algorithm', 'x1', 'x2', 'f', 'total_time', 'global_minimum']]
)

# For now let's remove sine-envelope function because there is something
# going on there
mask = deeplifting_results['problem_name'] != 'sine_envelope'
deeplifting_results = deeplifting_results.loc[mask].reset_index(drop=True)
deeplifting_results['hits'] = np.abs(
    deeplifting_results['global_minimum'] - deeplifting_results['f']
) <= 1e-4
deeplifting_results['hits'] = deeplifting_results['hits'].astype(int)
deeplifting_results = deeplifting_results.rename(columns={'total_time': 'time'})
deeplifting_results = deeplifting_results.drop(columns=['global_minimum'])
deeplifting_results.head()

In [ ]:
deeplifting_results.groupby('problem_name')['hits'].mean()

In [ ]:
# Load in the comparisons
comparison_files = glob.glob('./results/algorithm-comparisons-*.parquet')
comparison_results = pd.read_parquet(comparison_files)

# # Omit sine envelope function
# mask = comparison_results['problem_name'] != 'sine_envelope'
# comparison_results = comparison_results.loc[mask].reset_index(drop=True)
problems = [
    'ackley',
    'ackley2',
    'adjiman',
    'alpine1',
    'alpine2',
    'bartels_conn',
    'beale',
    'bird',
    'bukin_n2',
    'bukin_n4',
    'bukin_n6',
    'cross_in_tray',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
    'ex8_1_1',
    'kriging_peaks_red010',
    'mathopt6',
    'quantum',
    'rosenbrock',
]

comparison_results = comparison_results.loc[comparison_results['problem_name'].isin(problems)]

In [ ]:
comparison_results['problem_name'].value_counts()

In [ ]:
results_df = comparison_results.copy()

In [ ]:
results_df.loc[results_df['problem_name'] == 'ackley3']

In [ ]:
aggregated_results_df = (
    results_df
    .groupby(['problem_name', 'algorithm'])
    .agg({'hits': 'mean'})
    .rename(columns={'hits': 'Success Rate'})
    .unstack()
    .transpose()
    .reorder_levels(['algorithm', None])
    .transpose()
)

In [ ]:
aggregated_results_df.mean().sort_values(ascending=False)

In [ ]:
aggregated_results_df

In [ ]:
aggregated_results_df.shape

In [ ]:
file = [
    './results/results-2023-08-2-layer-512-holder_table-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-512-mathopt6-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-schwefel-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-shubert-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-512-cross_leg_table-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-ackley2-0-particle-leaky_relu.parquet',
]
df = pd.read_parquet(file)

In [ ]:
df['hits'] = np.abs(df['global_minimum'] - df['f']) <= 1e-4

In [ ]:
df.groupby('problem_name')['hits'].agg(['mean', 'count'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def ackley3(x, y):
    return -200 * np.exp(-0.02 * np.sqrt(x ** 2 + y ** 2)) + 5 * np.exp(np.cos(3 * x) + np.sin(3 * y))

x = np.linspace(-32, 32, 10000)
y = np.linspace(-32, 32, 10000)
X, Y = np.meshgrid(x, y)
Z = ackley3(X, Y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Plot of Ackley Function')

plt.show()

In [ ]:
ackley3(0, -0.40)

In [ ]:
np.min(Z)

In [54]:
problem = PROBLEMS_BY_NAME['cube']

In [55]:
problem

{'objective': <function deeplifting.problems.cube(x, results, trial, version='numpy')>,
 'bounds': [(-10, 10), (-10, 10)],
 'max_iterations': 1000,
 'global_minimum': 0,
 'dimensions': 2}

In [56]:
import numpy as np
import torch

In [57]:
results = np.zeros((1, 1, 3))
trial = 0
f = problem['objective']

In [62]:
values = [1.0, 1.0]
f(torch.tensor(values), results=results, trial=trial, version='pytorch')

tensor(0.)

In [63]:
f(np.array(values), results=results, trial=trial, version='numpy')

0.0